In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["ampform", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Standard lineshapes

AmpForm provides a few common lineshapes through the {mod}`.dynamics` module. This pages shows some of the properties of these lineshapes. It's also possible to insert {doc}`custom lineshapes </usage/dynamics/custom>` into the amplitude model.

In [ ]:
import sympy as sp
from IPython.display import Math

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from qrules.combinatorics import arange


def plot_real_imag(
    expression: sp.Expr,
    variable: sp.Symbol,
    x_min: float,
    x_max: float,
    resolution: int = 100,
) -> Figure:
    delta = (x_max - x_min) / resolution
    x = list(arange(x_min, x_max, delta))
    y_real = list(map(lambda x: sp.Abs(expression).subs(variable, x), x))
    y_imag = list(map(lambda x: sp.arg(expression).subs(variable, x), x))
    fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(8, 8))
    for a in ax:
        a.xaxis.set_ticks([])
        a.yaxis.set_ticks([])
    ax_real, ax_imag = ax
    ax_imag.set(xlabel=f"${variable.name}$")
    ax_imag.set(ylabel=f"imag $f({variable.name})$")
    ax_real.set(ylabel=f"real $f({variable.name})$")
    ax_real.yaxis.set_ticks([])
    ax_imag.yaxis.set_ticks([0, float(sp.pi)])
    ax_imag.yaxis.set_ticklabels([0, R"$\pi$"])
    ax_imag.set_ylim([0, float(sp.pi)])
    ax_real.plot(x, y_real)
    ax_imag.plot(x, y_imag)
    return fig

## Form factor

AmpForm uses {class}`.BlattWeisskopf` functions $B_L$ as _barrier factors_ (also called _form factors_):

In [ ]:
from ampform.dynamics import BlattWeisskopf

L = sp.Symbol("L", integer=True)
z = sp.Symbol("z", real=True)
ff2 = BlattWeisskopf(L, z) ** 2

In [ ]:
Math(f"{sp.latex(ff2)} = {sp.latex(ff2.doit())}")

## Relativistic Breit-Wigner

### _Without_ form factor

{func}`.relativistic_breit_wigner`:

In [ ]:
from ampform.dynamics import relativistic_breit_wigner

m, m0, w0 = sp.symbols("m, m0, Gamma0", real=True)
relativistic_breit_wigner(s=m ** 2, mass0=m0, gamma0=w0)

In [ ]:
plot_real_imag(
    relativistic_breit_wigner(s=m ** 2, mass0=1.0, gamma0=0.3),
    m,
    x_min=0,
    x_max=2,
);

### _With_ form factor

{func}`.breakup_momentum_squared`:

In [ ]:
from ampform.dynamics import breakup_momentum_squared

m, m_a, m_b = sp.symbols("m, m_a, m_b", real=True)
q_squared = breakup_momentum_squared(s=m ** 2, m_a=m_a, m_b=m_b)

In [ ]:
Math(f"q^2(m) = {sp.latex(q_squared)}")

{func}`.coupled_width`:

In [ ]:
from ampform.dynamics import coupled_width

L = sp.Symbol("L", integer=True)
m0, w0, d = sp.symbols("m0, Gamma, d", real=True)
s = m ** 2
running_width = coupled_width(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
)

In [ ]:
q_squared = breakup_momentum_squared(s, m_a, m_b)
q0_squared = breakup_momentum_squared(m0 ** 2, m_a, m_b)
q = sp.sqrt(q_squared)
q0 = sp.sqrt(q0_squared)
ff = BlattWeisskopf(L, z=q_squared * d ** 2)
ff0 = BlattWeisskopf(L, z=q0_squared * d ** 2)
running_width = running_width.subs(
    {
        2 * q: sp.Symbol("q(m)"),
        2 * q0: sp.Symbol("q(m_{0})"),
        ff: sp.Symbol("B_{L}(q)"),
        ff0: sp.Symbol("B_{L}(q_{0})"),
        sp.sqrt(s): m,
    }
)
Math(fR"\Gamma(m) = {sp.latex(running_width)}")

{func}`.relativistic_breit_wigner_with_ff`:

In [ ]:
from ampform.dynamics import relativistic_breit_wigner_with_ff

rel_bw_with_ff = relativistic_breit_wigner_with_ff(
    s=s,
    mass0=m0,
    gamma0=w0,
    m_a=m_a,
    m_b=m_b,
    angular_momentum=L,
    meson_radius=d,
)

In [ ]:
q_squared = breakup_momentum_squared(s, m_a, m_b)
ff = BlattWeisskopf(L, z=q_squared * d ** 2)
mass_dependent_width = coupled_width(s, m0, w0, m_a, m_b, L, d)
rel_bw_with_ff.subs(
    {
        2 * q_squared: 2 * sp.Symbol("q^{2}(m)"),
        ff: sp.Symbol(R"B_{L}\left(q\right)"),
        mass_dependent_width: sp.Symbol(R"\Gamma(m)"),
    }
)

In [ ]:
ma = 0.2
mb = 0.3
complex_bw_ff = relativistic_breit_wigner_with_ff(
    s=m ** 2,
    mass0=1.0,
    gamma0=0.3,
    m_a=ma,
    m_b=mb,
    angular_momentum=0,
    meson_radius=1,
)
plot_real_imag(complex_bw_ff.doit(), m, x_min=ma + mb, x_max=2);